In [1]:
import pandas as pd
import re
import API

In [2]:
!pip install openai


In [3]:
import openai



In [4]:
csv_files = [
    'US_INDU_INCOME_STATEMENT_KEY.csv',
    'US_INDU_INCOME_STATEMENT.csv',
    'US_INDU_CASH_FLOW_STATEMENT_KEY.csv',
    'US_INDU_CASH_FLOW_STATEMENT.csv',
    'US_INDU_CALCULATIONS_KEY.csv',
    'US_INDU_CALCULATIONS.csv',
    'US_INDU_BALANCE_SHEET_STATEMENT_KEY.csv',
    'US_INDU_BALANCE_SHEET_STATEMENT.csv'
]

data_frames = {}

for csv_file in csv_files:
    df_key = csv_file.replace('.csv', '').replace('US_INDU_', '').replace('_', ' ')
    data_frames[df_key] = pd.read_csv(csv_file)

In [5]:
openai.api_key = API.OPENAI_API_KEY

In [6]:
def answer_question(user_question: str) -> str:
    
    topics = ["Total Revenue", "Net Income", "Price to Earnings", "Dividend Yield", "Return on Equity"]
    response_topics = openai.Completion.create(
        engine="davinci",
        prompt="Given this data answer the following user question  Data: {data}  Question: " + user_question,
        max_tokens=50,
        temperature=0.6,
    )
    print("Response from OpenAI topics:", response_topics.choices[0].text)
    
    matched_topic = None
    
   
    for topic in topics:
        if topic.lower() in response_topics.choices[0].text.lower():
            matched_topic = topic
            break
    
    if matched_topic is None:
        return "I'm sorry, I couldn't understand the question."
    
    print("Matched Topic:", matched_topic)
    
    company_name = "Apple"
    fiscal_year = "2021"
    
   
    if matched_topic in ["Total Revenue", "Net Income"]:
        data_df = data_frames["INCOME STATEMENT"]
        quantity_column = "netincome"
    elif matched_topic == "Price to Earnings":
        data_df = data_frames["CALCULATIONS"]
        quantity_column = "pricetoearnings"
    elif matched_topic == "Dividend Yield":
        data_df = data_frames["CALCULATIONS"]
        quantity_column = "dividendyield"
    elif matched_topic == "Return on Equity":
        data_df = data_frames["CALCULATIONS"]
        quantity_column = "roe"
    else:
        return "I'm sorry, I couldn't understand the question."
    

    filtered_value = filter_data(data_df, company_name, fiscal_year, quantity_column)
    print("Filtered Value:", filtered_value)
    
    
    response_answer = openai.Completion.create(
        engine="davinci",
        prompt=f"Given this data answer the following user question ## Data: {data} ## Question: " + f"In {fiscal_year}, {company_name} {matched_topic} was {filtered_value}.",
        max_tokens=50,
        temperature=0.6,
    )
    final_response = response_answer.choices[0].text.strip()
    
    return final_response


In [7]:
user_question = "In 2021, how much profit did Apple generate from its shareholders' equity?"
response = answer_question(user_question)
print(response)


Response from OpenAI topics:  ____________________________________________________________ ____________________________________________________________ ____________________________________________________________ ____________________________________________________________ ____________________________________________________________ ____________________________________________________________ ____________________________________________________________ ____________________________________________________________ ____________________________________________________________ ____________________________________________________________ ____________________________________________________________ ____________________________________________________________ ________________________________
I'm sorry, I couldn't understand the question.
